In [3]:
%config IPCompleter.greedy=True
import os
import pygal
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import cairosvg
import datetime
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.stats import chisquare
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, roc_auc_score, roc_curve

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Lars, ElasticNet, Lasso, Ridge, BayesianRidge
pd.set_option('display.float_format', lambda x: "{:,.2f}".format(x))
path_datos = "/home/lalo/Documentos/Diplomado Ciencia de Datos/Modulo 2 Modelaciòn Supervisada/Bases de Datos/Wine quality/"

In [14]:
df_red = pd.read_csv(path_datos + "winequality-red.csv",sep = ';')
df_red_aux = pd.read_csv(path_datos + "winequality-red.csv",sep = ';')

df_white = pd.read_csv(path_datos + "winequality-white.csv",sep = ';')
df_white_aux = pd.read_csv(path_datos + "winequality-white.csv",sep = ';')

print(df_red.shape, df_white.shape)

(1599, 12) (4898, 12)


In [15]:
ls_pred = [x for x in df_white.columns]
ls_pred

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [17]:
df_white = pd.read_csv(path_datos + "winequality-white.csv",sep = ';')
df_white_aux = pd.read_csv(path_datos + "winequality-white.csv",sep = ';')

print(df_white.shape,df_white_aux.shape)

(4898, 12) (4898, 12)


In [18]:
def New_pob_var_sigma(data_origen,var, enes=3):
    data = pd.DataFrame(data_origen)
    
    sigma = data[var].std()
    media = data[var].mean()
    lim_inf  = media - enes*sigma
    lim_sup = media + enes*sigma
    
    return lim_inf,lim_sup

In [19]:
for var_cont_i in ls_pred:
# Se calculan las cotas superiores e inferiores de cada variable de la tabla original  (df)
# Asì no afectamos que se haya reducido la poblaciòn
    Cota_Inf, Cota_Sup = New_pob_var_sigma(df_white,var_cont_i,4)
    
# Se reducirà un dataframe auxiliar, tomando en cuenta los limites del dataframe original
    df_white_aux = df_white_aux[(Cota_Inf <= df_white_aux[var_cont_i]) & (df_white_aux[var_cont_i] <= Cota_Sup)]
  

df_white_aux.shape

(4739, 12)

In [20]:
df_red = pd.read_csv(path_datos + "winequality-red.csv",sep = ';')
df_red_aux = pd.read_csv(path_datos + "winequality-red.csv",sep = ';')

print(df_red.shape,df_red_aux.shape)

(1599, 12) (1599, 12)


In [21]:
for var_cont_i in ls_pred:
# Se calculan las cotas superiores e inferiores de cada variable de la tabla original  (df)
# Asì no afectamos que se haya reducido la poblaciòn
    Cota_Inf, Cota_Sup = New_pob_var_sigma(df_red,var_cont_i,4)
    
# Se reducirà un dataframe auxiliar, tomando en cuenta los limites del dataframe original
    df_red_aux = df_red_aux[(Cota_Inf <= df_red_aux[var_cont_i]) & (df_red_aux[var_cont_i] <= Cota_Sup)]
  

df_red_aux.shape

(1537, 12)

In [6]:
var_object = "quality"

Index(['fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"'], dtype='object')

In [7]:
var_object = "quality"

In [11]:
df_red.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5
1,7.80,0.88,0.00,2.60,0.10,25.00,67.00,1.00,3.20,0.68,9.80,5
2,7.80,0.76,0.04,2.30,0.09,15.00,54.00,1.00,3.26,0.65,9.80,5
3,11.20,0.28,0.56,1.90,0.07,17.00,60.00,1.00,3.16,0.58,9.80,6
4,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5
5,7.40,0.66,0.00,1.80,0.07,13.00,40.00,1.00,3.51,0.56,9.40,5
6,7.90,0.60,0.06,1.60,0.07,15.00,59.00,1.00,3.30,0.46,9.40,5
7,7.30,0.65,0.00,1.20,0.07,15.00,21.00,0.99,3.39,0.47,10.00,7
8,7.80,0.58,0.02,2.00,0.07,9.00,18.00,1.00,3.36,0.57,9.50,7
9,7.50,0.50,0.36,6.10,0.07,17.00,102.00,1.00,3.35,0.80,10.50,5


In [13]:
def plot_histogram(data,var):
    path_img_hist = "/home/lalo/Documentos/Diplomado Ciencia de Datos/Modulo 2 Modelaciòn Supervisada/Ejercicios En Equipo/practica 2/"
    ye_s, equis = np.histogram(data[var])
    hist = list(zip(ye_s, equis, equis[1:]))
    hist_plot = pygal.Histogram()
    hist_plot.add(var + "_dist", hist)
    hist_plot.render_to_png(path_img_hist + var)

In [ ]:
ls_pred = [x for x in df_white.columns if x not in [tgt]]

In [ ]:
for var_cont in ls_cont:
   [plot_histogram(train,var_cont)]